# Example of how to download particle information 
# from TNG50 to run the iMaStar and iMaNGA codes

In [1]:
import numpy as np

import astropy.cosmology as co
cosmo = co.Planck15
import os
import astropy.constants as constants
import astropy.units as uu

In [2]:
import API_functions as API


API.headers['api-key']='b65ee99582d40446ede7aa5ed7d79ac4'

## Define which galaxy you want to observe

### https://www.tng-project.org/api/ use the API
### to investigate the properties of snapshots and subhaloes

##### N.B. in Illustris vocabulary subhalo = "galaxy"; halo = "cluster"
##### N.B. Since the definition of "galaxy" in the simulation is not an astrophysics one
##### to use them as "galaxies"  you need to check if they have more then 10'000 stellar particles
##### and cut to outskirt particles (see Fig. 2 in Nanni+2022)

In [3]:
snap = 33 # this corrisponds to redshift 2
gal = 20 # this is the 20th most massive galaxy in the snapshot

z = 2

namegal = str('snap'+str(snap)+"gal"+str(gal))

## Stellar particle information

In [4]:
fields=[
    [4,'Coordinates'],
    [4,'GFM_InitialMass'],
    [4,'GFM_Metallicity'],
    [4,'GFM_StellarFormationTime'],
    [4,'Velocities']

]

# definition of units

kpc = float(np.asarray(constants.kpc))
h = float(np.asarray(cosmo.h))
SolarMass = float(np.asarray(constants.M_sun))
scale_factor = 1/(1+z)
yr_sec = 3.154e+7

In [5]:
data = API.getGalaxy(gal,fields, simulation='TNG50-1', snapshot=snap)

In [6]:
coordinate = data[0][:,:]*scale_factor/h #stellar particle position in 
# the simulation box [kpc]

mass = data[1][:]*10**10/h # [M_sun]
metallicity = data[2][:] # Z
SFT = data[3][:] # a at the time of the stellar particle formation
vel = data[4][:,:]/np.sqrt(scale_factor) # stellar particle velocity
#in the simulation box[km/s]


In [7]:
# This can be used as an example of how to obtain other information from TNG50

# For iMaNGA, I constructed catalogue files with the info of the galaxies we wanted to observe
# Here, I commented out the different entry just because it takes a bit of time

In [8]:
# position of the centre of mass of the galaxy:
SHP = API.getSubhaloField('SubhaloPos', simulation='TNG50-1', snapshot=snap)[gal]/h

# velocity of the centre of mass of the galaxy:
subhalovelocity = API.getSubhaloField('SubhaloVel', simulation='TNG50-1', \
                                      snapshot=snap)[gal]/np.sqrt(scale_factor)

# mass of the galaxy:
#SubhaloMass = API.getSubhaloField('SubhaloMass', simulation='TNG50-1', snapshot=snap)[gal]*cosmo.h

#SFR of the galaxy:
#SubhaloSFR = API.getSubhaloField('SubhaloSFR', simulation='TNG50-1', snapshot=snap)[gal]

#half mass stellar radius of the galaxy:
HMSR = API.getSubhaloField('SubhaloHalfmassRadType',snapshot=snap,simulation='TNG50-1')[gal, 4]/h 

#U, B, V, K, g, r, i, z.

#SubhaloStellarPhotometrics_i = iApi.getSubhaloField('SubhaloStellarPhotometrics',\
# simulation='TNG50-1', snapshot=snap)[gal, 6]

In [9]:
# we assume the centre of mass of the galaxy as frame of reference 
starx = coordinate[:,0] - SHP[0] 
stary = coordinate[:,1] - SHP[1]
starz = coordinate[:,2] - SHP[2]

#computing the radius
RadiusStar = np.sqrt((starx)**2 + (stary)**2 + (starz)**2)


In [11]:
# the following step is necessary to go from the moment 
# of the stellar birth given as a
# to the age of the stars in the moment of the observation 

RedStar = (1-SFT)/SFT

eta = cosmo.lookback_time(RedStar)
eta_galaxy = np.asarray(cosmo.lookback_time(z))

Eta = np.asarray(eta) - eta_galaxy

In [12]:
# we go from Z to [Z/H]
metallicita = np.log10(metallicity/0.018)

# we cut stellar particles outside 15 times the half_mass_stellar_radius
StarPosition = coordinate[np.where((SFT > 0) & (RadiusStar < 15.*HMSR)) ] - SHP
StarMass = mass[np.where((SFT > 0) & (RadiusStar < 15.*HMSR)) ]
StarMet = metallicita[np.where((SFT > 0) & (RadiusStar < 15.*HMSR)) ]
Age = Eta[np.where((SFT > 0) & (RadiusStar < 15.*HMSR)) ]
Vel = vel[np.where((SFT > 0) & (RadiusStar < 15.*HMSR)) ] - subhalovelocity

h_star = np.zeros(np.shape(StarMass))


In [13]:
# here we distinguish between young and old stellar particles, you can pick your own cut

# young stellar particles, i.e. <1 Myr here: 
array1 = StarPosition[:,0][Age < 10**-3 ]*10**3 #[pc]
array2 = StarPosition[:,1][Age < 10**-3 ]*10**3 #[pc]
array3 = StarPosition[:,2][Age < 10**-3 ]*10**3 #[pc]

array4 = h_star[Age < 10**-3 ] #smoothing parameter
array5 = StarMass[Age < 10**-3 ]*10**-7 # computed the [SFR]
array6 = 10**StarMet[Age < 10**-3 ] # [Z/H]
array7 = 5.*np.ones(np.shape(array6)) # logC for MappingsIII
array8 = 5.*np.ones(np.shape(array6)) #logp for MappingsIII
array9 = 0.1*np.ones(np.shape(array6)) #fPDR for MappingsIII
array10 = Vel[:, 0][Age < 10**-3 ] #[km/s]
array11 = Vel[:, 1][Age < 10**-3 ] #[km/s]
array12 = Vel[:, 2][Age < 10**-3 ] #[km/s]
array13 = StarMass[Age < 10**-3 ] # [M_sun]
array14 = StarMet[Age < 10**-3 ] # [Z/H]
array15 = Age[:][Age < 10**-3 ] #[Gyr]



In [14]:
# old stellar particles, i.e. >1 Myr here: 

aarray1 = StarPosition[:,0][Age > 10**-3]*10**3 #[pc]
aarray2 = StarPosition[:,1][Age > 10**-3]*10**3 #[pc]
aarray3 = StarPosition[:,2][Age > 10**-3]*10**3 #[pc]
aarray4 = h_star[Age > 10**-3] #smoothing parameter
aarray5 = StarMass[Age > 10**-3] # [M_sun]
aarray6 = StarMet[Age > 10**-3] # [Z/H]
aarray7 = Age[:][Age > 10**-3] #[Gyr]
aarray8 = Vel[:, 0][Age > 10**-3] #stellar particle x velocity [km/s]
aarray9 = Vel[:, 1][Age > 10**-3] #stellar particle y velocity[km/s]
aarray10 = Vel[:, 2][Age > 10**-3] #stellar particle z velocity [km/s]
aarray11 = np.ones(len(aarray10))*subhalovelocity[0] #subhalo x velocity [km/s]
aarray12 = np.ones(len(aarray10))*subhalovelocity[1] #subhalo y velocity [km/s]
aarray13 = np.ones(len(aarray10))*subhalovelocity[2] #subhalo z velocity [km/s]


In [15]:
# construction of the files

In [16]:
os.mkdir("./"+namegal)

In [17]:
with open("./"+namegal+"/"+namegal+'.dat', 'w') as f:
    for a, b, c, d, e, ss, tt in zip(aarray1, aarray2, \
                                     aarray3, aarray4, aarray5, aarray6, aarray7):
        f.write('{0:} {1:} {2:} {3:} {4:} {5:} {6:}\n'.format(a, b, c, d, e, ss, tt))

with open("./"+namegal+"/"+namegal+'velocity.dat', 'w') as f:
    for a, b, c, d, e,g in zip(aarray8, aarray9, aarray10, aarray11, aarray12, aarray13):
        f.write('{0:} {1:} {2:} {3:} {4:} {5:}\n'.format(a, b, c, d, e, g))



if len(array1)>1:
    with open("./"+namegal+"/"+namegal+'MIII.dat', 'w') as f:
        for a, b, c, d, e, ss, tt, r, s in zip( array1, array2, array3, \
                                               array4, array5, array6, array7, array8, array9):
            f.write('{0:} {1:} {2:} {3:} {4:} {5:} {6:} {7:} {8:}\n'.format(a, b, c, d, e, ss, tt, r, s))

    with open("./"+namegal+"/"+namegal+'velocityMIII.dat', 'w') as f:
        for a, b, c in zip(array10, array11, array12):
            f.write('{0:} {1:} {2:}\n'.format(a, b, c))
            
    with open("./"+namegal+"/"+namegal+'starsMIII.dat', 'w') as f:
        for a, b, c in zip(array13, array14, array15):
            f.write('{0:} {1:} {2:}\n'.format(a, b, c))


## Gas particles information

In [20]:
fields_dust=[
    [0,'Coordinates'],
    [0, 'Masses'],
    [0,'GFM_Metallicity'],
    [0,'InternalEnergy'],
    [0,'ElectronAbundance'],
    [0,'StarFormationRate'], 
    [0, 'Velocities']
]


data_dust = API.getGalaxy(gal,fields_dust, simulation='TNG50-1', snapshot=snap)


In [ ]:
#gas particle information:

In [21]:
coordinategas = data_dust[0][:,:]*scale_factor/h  #[kpc] 
massgas = data_dust[1][:]*10**10/h #M_sun
metallicitygas = data_dust[2][:] # Z
Energy = data_dust[3][:]
ElecAbb = data_dust[4][:]
SFRgas = data_dust[5][:]
vel_gas = data_dust[6][:,:]/np.sqrt(scale_factor) #[km/s]

RadiusDustC = ((coordinategas[:,0]-SHP[0])**2+\
               (coordinategas[:,1]-SHP[1])**2+(coordinategas[:,2]-SHP[2])**2)

RadiusDust = np.sqrt(RadiusDustC)
VelGas = vel_gas - subhalovelocity

Xh=0.76
gamma=5./3.
mp=1.6726e-24 # g
kb= 1.3807e-16

# computing the dust temperature 
T=(gamma-1.)*(Energy/kb)*(4./(1.+3.*Xh+4.*Xh*ElecAbb))*mp*10**10


In [22]:
Dust_pos_x = (coordinategas[:,0]-SHP[0])*10**3 #pc
Dust_pos_y = (coordinategas[:,1]-SHP[1])*10**3 #pc
Dust_pos_z = (coordinategas[:,2]-SHP[2])*10**3 #pc

# cutting particles outside 15HMSR
dust_x = Dust_pos_x[ (SFRgas>0.) & (RadiusDust<15.*HMSR)]
dust_y = Dust_pos_y[ (SFRgas>0.) & (RadiusDust<15.*HMSR)]
VelGas = VelGas[ (SFRgas>0.) & (RadiusDust<15.*HMSR)]

aarray6 = dust_x
aarray7 = dust_y
aarray8 = VelGas[:, 0]
aarray9 = VelGas[:, 1]
aarray10 = VelGas[:, 2]

In [23]:
with open("./"+namegal+"/"+namegal+'velocityGas.dat', 'w') as f:
    for a, b, c, d,e in zip(aarray6, aarray7, aarray8, aarray9, aarray10):
        f.write('{0:} {1:} {2:} {3:} {4:}\n'.format(a, b, c, d,e))


In [24]:
from scipy import spatial
from sklearn.neighbors import KDTree

In [25]:
# here we remuve the dast particles in the same region as MIII regions
# since MIII alread account for dust

if len(array1)>1:
    from scipy import spatial
    gas_position = np.column_stack((Dust_pos_x,Dust_pos_y, Dust_pos_z))
    miii_position = np.column_stack((array1,array2, array3))

    tree = spatial.KDTree(gas_position)
    gas_id = tree.query_ball_point(miii_position, r=100)
    
    gas_id_arr = []
    for i in range(len(gas_id)):
        for j in range(len(gas_id[i])):
            gas_id_arr.append(gas_id[i][j])
            
    massgas = np.delete(massgas, gas_id_arr)
    metallicitygas = np.delete(metallicitygas, gas_id_arr)
    T = np.delete(T , gas_id_arr)
    coordinategas = np.delete(coordinategas, gas_id_arr, axis=0)
    RadiusDust = np.delete(RadiusDust, gas_id_arr)
    SFRgas = np.delete(SFRgas, gas_id_arr)
    

In [26]:
DustPosit = coordinategas[np.where((RadiusDust<15.*HMSR)& (np.logical_or(SFRgas>0,T<=8000 ))) ] - SHP
DustMasse = massgas[np.where((RadiusDust<15.*HMSR)& (np.logical_or(SFRgas>0,T<=8000 ))) ]
DustZ = metallicitygas[np.where((RadiusDust<15.*HMSR)& (np.logical_or(SFRgas>0,T<=8000 ))) ]
DustT = T[np.where((RadiusDust<15.*HMSR)& (np.logical_or(SFRgas>0,T<=8000 ))) ]

if len(DustZ)>100: # since running RT is really heavy, 
    #it is better to put a cut on how many particles you want before running it, to not run it for no real reason
	Tree = np.array(DustPosit)
	kdt = KDTree(Tree, leaf_size=30, metric='euclidean')
	Treedx, ndx = kdt.query(Tree, k=8, return_distance=True)
	hdust = Treedx[:,7]*10**3
	array1 = DustPosit[:,0]*10**3
	array2 = DustPosit[:,1]*10**3
	array3 = DustPosit[:,2]*10**3
	array4 = np.asarray(hdust)
	array5 = DustMasse
	array6 = np.asarray(DustZ)
	array7 = np.asarray(DustT)
	if len(array1)>0:
    		with open("./"+namegal+"/"+namegal+'dust.dat', 'w') as f:
        		for a, b, c, d, e, ff, gg in zip( array1, array2, array3, array4, array5, array6, array7):
            			f.write('{0:} {1:} {2:} {3:} {4:} {5:} {6:}\n'.format(a, b, c, d, e, ff, gg))




